# __*Tugas Final Mata Kuliah Data Mining Kelompok 21*__

**Anggota kelompok:**

* Muhammad Fajri Rasid (H071191051)
* Muh. Yusuf Syam (H071191044)
* Nur Aqila (H071201088)
* Muhammad Amirul Haq (H071201079)
* Siskanti (H071201064)

Download dataset

In [ ]:
!pip install kaggle

import os

# Mengatur environment username dan key dari API yang telah disediakan oleh kaggle
os.environ['KAGGLE_USERNAME'] = "###Username Kaggle###"
os.environ['KAGGLE_KEY'] = "###Token Kaggle###"

# Download data dari kaggle
!kaggle competitions download -c DM2021MidFinalSisfoUnhas

# Karena beberapa file berupa archive, maka perlu di unzip terlebih dahulu
!unzip -q test.csv.zip
!unzip -q train.csv.zip

  0% 0.00/1.49M [00:00<?, ?B/s]
100% 1.49M/1.49M [00:00<00:00, 49.6MB/s]
  0% 0.00/108k [00:00<?, ?B/s]
100% 108k/108k [00:00<00:00, 109MB/s]
  0% 0.00/648k [00:00<?, ?B/s]
100% 648k/648k [00:00<00:00, 89.2MB/s]


Import library-library yang dibutuhkan

In [ ]:
# Pengolahan data
import pandas as pd
import numpy as np
from collections import Counter
from datetime import date

# Preprocessing data
from sklearn.preprocessing import StandardScaler

# Classification model
from xgboost import XGBClassifier

# __Membangun Model__

Membaca data train lalu menampilkan 5 data pertama

In [ ]:
df = pd.read_csv('train.csv').iloc[:, 1:]

df.head()

,gender,date_of_birth,workclass,education_level,education_num,marital_status,occupation,relationship,capital_gain,capital_loss,hours_per_week,approved,address,email,inquiry_purpose_code,institute_type,account_type,asset_class_cd,asset_code,portfolio_type
0,Female,1950-01-12,Local-gov,HS-grad,9.0,Widowed,Adm-clerical,Not-in-family,0.0,0.0,12.0,0,"9767 Snyder Summit Suite 944\r\nWest Jesse, FL...",erinfox@jones.info,16.0,NBF,5,NaN,1.0,R
1,Male,1965-01-08,Private,Assoc-voc,11.0,Married-civ-spouse,Machine-op-inspct,Husband,0.0,0.0,20.0,0,"89313 Christina Divide\r\nGarciatown, ME 71630",ykoch@gmail.com,2.0,PUB,47,S,1.0,I
2,Female,1997-12-31,Private,Assoc-voc,11.0,Never-married,Priv-house-serv,Not-in-family,0.0,0.0,30.0,0,"896 Benjamin Shore Suite 035\r\nSamuelhaven, T...",wolfekayla@mills.com,13.0,PVT,189,NaN,1.0,I
3,Male,1963-01-09,Local-gov,Some-college,10.0,Widowed,Exec-managerial,Not-in-family,6849.0,0.0,40.0,0,"706 Crystal Roads\r\nNorth Marcus, NE 73557",ejackson@taylor.net,16.0,NBF,5,NaN,1.0,R
4,Female,1996-01-01,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Other-relative,0.0,0.0,20.0,1,"821 Sara Falls Apt. 775\r\nSouth Sarah, AZ 45554",cannonshawn@strong.com,99.0,BRO,5,NaN,1.0,R


Menampilkan bentuk/shape serta deskripsi statistik dari data train

In [ ]:
df.shape

(31655, 20)

In [ ]:
df.describe()

,education_num,capital_gain,capital_loss,hours_per_week,approved,inquiry_purpose_code,account_type,asset_code
count,31655.000000,31655.000000,31655.000000,31655.000000,31655.000000,30738.000000,31655.000000,31592.000000
mean,10.117422,1088.854020,90.434434,40.977002,0.247860,29.024953,80.390523,1.064352
std,2.558635,7417.324283,408.651402,11.968719,0.431777,35.437741,145.927065,0.716770
min,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,5.000000,1.000000
25%,9.000000,0.000000,0.000000,40.000000,0.000000,13.000000,5.000000,1.000000
50%,10.000000,0.000000,0.000000,40.000000,0.000000,13.000000,5.000000,1.000000
75%,13.000000,0.000000,0.000000,45.000000,0.000000,16.000000,173.000000,1.000000
max,16.000000,99999.000000,4356.000000,99.000000,1.000000,99.000000,999.000000,19.000000


## __Preprocessing__

Karena tahap preprocessing pada data train dan data test dilakukan secara terpisah, maka beberapa tahap preprocessing dibuat dalam bentuk fungsi untuk memudahkan pengerjaan dan mengurangi penulisan kode yang berulang

Pertama-tama kita membuat variabel baru untuk preprocessing

In [ ]:
df_preprocess = df.copy()

#### __1. Data Wrangling__

Pada tahap ini kita membersihkan serta merapikan data. Karena data tidak terlalu berantakan, maka pada tahapan ini data tidak terlalu terpengaruh. 
<br>
<br>
Proses data wrangling meliputi: 
* __Strip pada string__

  Menghapus spasi di awal maupun di akhir pada setiap string di dataframe.
* __Mengubah tipe data kolom tertentu menjadi tipe data objek__

  Beberapa kolom bertipe data numerik namun sebenarnya bersifat kategorik, maka kita perlu mengubah tipe data kolom tersebut menjadi object (dtype untuk data kategorik) agar kolom-kolom tersebut ditangani sebagai kolom kategorik.
* __Mengurutkan ulang kolom target__ 

  Menyimpan kolom target pada akhir dataframe.
* __Membersihkan data kategorik yang bersifat outlier__

  Pada kolom asset_class_cd terdapat data yang bernilai '01', '?', 'nan', dan '-1' sedangkan tipe kelas asset seharusnya hanya berjumlah 5 yaitu: S, M, B, L, dan D.

In [ ]:
# Fungsi untuk strip string
def stripping(df):
  for i in df.select_dtypes(object):
    df[i] = df[i].astype('str').str.strip().astype(object)

# Fungsi untuk mengubah dtype suatu kolom menjadi dtype object
def col_to_object(df, *col):
  for i in col:
    df[i] = df[i].astype(object)

# Fungsi untuk mengurutkan ulang kolom target
def target_to_last(df, target_col='approved'):
  target = df[target_col]
  
  df.drop(target_col, axis='columns', inplace=True)

  df[target_col] = target

# Fungsi untuk membersihkan kolom asset_class_cd
def asset_class_cd_cleaning(df):
  df['asset_class_cd'] = df['asset_class_cd'].replace(['01','?','nan','-1'], np.nan)

Setelah membuat semua fungsi data wrangling, semua fungsi tersebut dirangkum dalam satu fungsi baru.

Kita memberi kondisi, jika train=True atau jika dataframe yang dimasukkan dalam fungsi adalah data train, maka akan dilakukan pengurutan ulang kolom target, jika train=False atau data merupakan data test, maka tidak akan dilakukan.

In [ ]:
def data_wrangling(df, train=True):
  stripping(df)
  col_to_object(df, 'account_type', 'inquiry_purpose_code', 'asset_code')
  asset_class_cd_cleaning(df)

  if(train):
    target_to_last(df)

Mengaplikasikan proses data wrangling pada df_preprocess, lalu menampilkan 5 data pertama

In [ ]:
data_wrangling(df_preprocess)

df_preprocess.head()

,gender,date_of_birth,workclass,education_level,education_num,marital_status,occupation,relationship,capital_gain,capital_loss,hours_per_week,address,email,inquiry_purpose_code,institute_type,account_type,asset_class_cd,asset_code,portfolio_type,approved
0,Female,1950-01-12,Local-gov,HS-grad,9.0,Widowed,Adm-clerical,Not-in-family,0.0,0.0,12.0,"9767 Snyder Summit Suite 944\r\nWest Jesse, FL...",erinfox@jones.info,16,NBF,5,NaN,1,R,0
1,Male,1965-01-08,Private,Assoc-voc,11.0,Married-civ-spouse,Machine-op-inspct,Husband,0.0,0.0,20.0,"89313 Christina Divide\r\nGarciatown, ME 71630",ykoch@gmail.com,2,PUB,47,S,1,I,0
2,Female,1997-12-31,Private,Assoc-voc,11.0,Never-married,Priv-house-serv,Not-in-family,0.0,0.0,30.0,"896 Benjamin Shore Suite 035\r\nSamuelhaven, T...",wolfekayla@mills.com,13,PVT,189,NaN,1,I,0
3,Male,1963-01-09,Local-gov,Some-college,10.0,Widowed,Exec-managerial,Not-in-family,6849.0,0.0,40.0,"706 Crystal Roads\r\nNorth Marcus, NE 73557",ejackson@taylor.net,16,NBF,5,NaN,1,R,0
4,Female,1996-01-01,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Other-relative,0.0,0.0,20.0,"821 Sara Falls Apt. 775\r\nSouth Sarah, AZ 45554",cannonshawn@strong.com,99,BRO,5,NaN,1,R,1


#### __2. Handling Missing Value__

Di sini, kita akan menghapus/drop baris dengan nilai nan

Mengecek kolom apa saja yang memiliki nilai nan

In [ ]:
df_preprocess.isnull().sum()

gender                      0
date_of_birth               0
workclass                   0
education_level             0
education_num               0
marital_status              0
occupation                  0
relationship                0
capital_gain                0
capital_loss                0
hours_per_week              0
address                     0
email                       0
inquiry_purpose_code      917
institute_type              0
account_type                0
asset_class_cd          25235
asset_code                 63
portfolio_type              0
approved                    0
dtype: int64

Terdapat 4 kolom dengan nilai nan, kolom porfolio_type juga memiliki nilai nan, namun nilai nan tersebut telah menjadi string 'nan' saat proses stripping string dilakukan, maka dari itu kita akan menggunakan teknik yang berbeda saat menangani kolom portfolio_type

In [ ]:
columns_with_nan_to_drop = ['inquiry_purpose_code','asset_code']

df_preprocess = df_preprocess[df_preprocess['portfolio_type'] != 'nan']
df_preprocess = df_preprocess.dropna(subset=columns_with_nan_to_drop).reset_index(drop=True)

len(df_preprocess)

30672

In [ ]:
df_preprocess

,gender,date_of_birth,workclass,education_level,education_num,marital_status,occupation,relationship,capital_gain,capital_loss,hours_per_week,address,email,inquiry_purpose_code,institute_type,account_type,asset_class_cd,asset_code,portfolio_type,approved
0,Female,1950-01-12,Local-gov,HS-grad,9.0,Widowed,Adm-clerical,Not-in-family,0.0,0.0,12.0,"9767 Snyder Summit Suite 944\r\nWest Jesse, FL...",erinfox@jones.info,16,NBF,5,NaN,1,R,0
1,Male,1965-01-08,Private,Assoc-voc,11.0,Married-civ-spouse,Machine-op-inspct,Husband,0.0,0.0,20.0,"89313 Christina Divide\r\nGarciatown, ME 71630",ykoch@gmail.com,2,PUB,47,S,1,I,0
2,Female,1997-12-31,Private,Assoc-voc,11.0,Never-married,Priv-house-serv,Not-in-family,0.0,0.0,30.0,"896 Benjamin Shore Suite 035\r\nSamuelhaven, T...",wolfekayla@mills.com,13,PVT,189,NaN,1,I,0
3,Male,1963-01-09,Local-gov,Some-college,10.0,Widowed,Exec-managerial,Not-in-family,6849.0,0.0,40.0,"706 Crystal Roads\r\nNorth Marcus, NE 73557",ejackson@taylor.net,16,NBF,5,NaN,1,R,0
4,Female,1996-01-01,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Other-relative,0.0,0.0,20.0,"821 Sara Falls Apt. 775\r\nSouth Sarah, AZ 45554",cannonshawn@strong.com,99,BRO,5,NaN,1,R,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30667,Female,1991-01-02,Private,Some-college,10.0,Divorced,Other-service,Unmarried,0.0,0.0,21.0,"0373 Pearson Summit Suite 375\r\nMendozashire,...",heather34@barrett-snyder.com,13,NBF,189,NaN,1,I,0
30668,Male,2002-12-30,Self-emp-not-inc,11th,7.0,Never-married,Farming-fishing,Own-child,0.0,0.0,15.0,1113 Christian Club Apt. 890\r\nChristopherbur...,michelle56@nelson-benitez.org,13,PVT,5,NaN,1,R,0
30669,Male,1962-01-09,Private,9th,5.0,Married-civ-spouse,Handlers-cleaners,Husband,0.0,0.0,40.0,"7492 Blake Pines\r\nLake Paul, NY 11187",pamela27@mathews.com,7,PVT,5,NaN,1,R,0
30670,Male,1937-01-15,Self-emp-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,0.0,2392.0,55.0,"400 Patterson Pine\r\nNorth Alexa, MA 30048",garciaedward@beltran.biz,99,NBF,47,S,1,I,1


#### __3. Feature Selection__

Membuat fungsi yang menghapus kolom-kolom tertentu yang tidak berkorelasi dengan kolom target

In [ ]:
def feature_selection(df, cols=['education_level', 'address', 'email']):
  df.drop(cols, axis='columns', inplace=True)

In [ ]:
feature_selection(df_preprocess)

df_preprocess.shape

(30672, 17)

#### __4. Feature Extraction__

* __Age__

Mendapatkan umur dalam hari berdasarkan tanggal lahir

In [ ]:
today = date.today()

def counting_birthday(x):
  global today
  year, month, day= [int(i) for i in x.split('-')]

  return (today - date(year, month, day)).days

* __Hours per week__

Mengelompokkan hours per week menjadi beberapa kategori

In [ ]:
def hours_per_week_map(x):
  x = int(x)

  if(x <= 40):
    return 'less than 40'
  elif(x <= 45):
    return 'between 40 and 45'
  elif(x <= 60):
    return 'between 45 and 60'
  elif(x <= 80):
    return 'between 60 and 80'
  else:
    return 'more than 80'

* __Marital status__

Mengelompokkan marital status menjadi 2, apakah sedang sendiri atau sedang mempunyai pasangan

In [ ]:
def marital_status_map(x):
    if(x in ['Divorced', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed']):
        return 'Single'
    else:
        return 'Couple'

Setelah itu mengaplikasikan semua fungsi 

In [ ]:
def feature_extraction(df):
  df.insert(1, 'age_(day)', df['date_of_birth'].map(counting_birthday))
  df.drop('date_of_birth', axis='columns', inplace=True)

  df['hours_per_week'] = df['hours_per_week'].map(hours_per_week_map)

  df['marital_status'] = df['marital_status'].map(marital_status_map)

  df.insert(7, 'capital_total', df['capital_gain'] - df['capital_loss'])
  df.drop(['capital_gain', 'capital_loss'], axis='columns', inplace=True)

In [ ]:
feature_extraction(df_preprocess)

df_preprocess.head()

,gender,age_(day),workclass,education_num,marital_status,occupation,relationship,capital_total,hours_per_week,inquiry_purpose_code,institute_type,account_type,asset_class_cd,asset_code,portfolio_type,approved
0,Female,26254,Local-gov,9.0,Single,Adm-clerical,Not-in-family,0.0,less than 40,16,NBF,5,NaN,1,R,0
1,Male,20779,Private,11.0,Couple,Machine-op-inspct,Husband,0.0,less than 40,2,PUB,47,S,1,I,0
2,Female,8734,Private,11.0,Single,Priv-house-serv,Not-in-family,0.0,less than 40,13,PVT,189,NaN,1,I,0
3,Male,21509,Local-gov,10.0,Single,Exec-managerial,Not-in-family,6849.0,less than 40,16,NBF,5,NaN,1,R,0
4,Female,9464,Self-emp-not-inc,13.0,Couple,Prof-specialty,Other-relative,0.0,less than 40,99,BRO,5,NaN,1,R,1


#### __5. Categorical Encoding__

Melakukan categorical encoding dengan teknik one hot encode

In [ ]:
def encoded(df):
  return pd.get_dummies(df)

In [ ]:
df_preprocess = encoded(df_preprocess)
target_to_last(df_preprocess)

df_preprocess

,age_(day),education_num,capital_total,gender_Female,gender_Male,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital_status_Couple,marital_status_Single,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,hours_per_week_between 40 and 45,hours_per_week_between 45 and 60,hours_per_week_between 60 and 80,hours_per_week_less than 40,hours_per_week_more than 80,inquiry_purpose_code_1.0,...,account_type_173,account_type_175,account_type_176,account_type_177,account_type_178,account_type_179,account_type_181,account_type_184,account_type_185,account_type_187,account_type_189,account_type_191,account_type_195,account_type_197,account_type_198,account_type_213,account_type_220,account_type_221,account_type_222,account_type_223,account_type_224,account_type_226,account_type_227,account_type_228,account_type_240,account_type_999,asset_class_cd_B,asset_class_cd_D,asset_class_cd_L,asset_class_cd_M,asset_class_cd_S,asset_code_1.0,asset_code_2.0,asset_code_4.0,asset_code_19.0,portfolio_type_F,portfolio_type_I,portfolio_type_M,portfolio_type_R,approved
0,26254,9.0,0.0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,20779,11.0,0.0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0
2,8734,11.0,0.0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
3,21509,10.0,6849.0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
4,9464,13.0,0.0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30667,11289,10.0,0.0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
30668,6909,7.0,0.0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
30669,21874,5.0,0.0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
30670,30999,13.0,-2392.0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1


#### __6. Normalisasi__

Melakukan normalisasi dengan z-score

In [ ]:
scaler = StandardScaler()

def normalized(df, train=True):
  global scaler
  
  features = df.copy()

  if(train):
    features.drop('approved', axis='columns', inplace=True)
    scaler.fit(features)
  
  scaled_features = scaler.transform(features)

  column_list = df.columns

  if(train):
    df = pd.concat([pd.DataFrame(data=scaled_features), df['approved']], axis='columns')
  else:
    df = pd.DataFrame(data=scaled_features)

  df.columns = column_list

  return df

In [ ]:
df_preprocess = normalized(df_preprocess, train=True)

df_preprocess.head()

,age_(day),education_num,capital_total,gender_Female,gender_Male,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital_status_Couple,marital_status_Single,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,hours_per_week_between 40 and 45,hours_per_week_between 45 and 60,hours_per_week_between 60 and 80,hours_per_week_less than 40,hours_per_week_more than 80,inquiry_purpose_code_1.0,...,account_type_173,account_type_175,account_type_176,account_type_177,account_type_178,account_type_179,account_type_181,account_type_184,account_type_185,account_type_187,account_type_189,account_type_191,account_type_195,account_type_197,account_type_198,account_type_213,account_type_220,account_type_221,account_type_222,account_type_223,account_type_224,account_type_226,account_type_227,account_type_228,account_type_240,account_type_999,asset_class_cd_B,asset_class_cd_D,asset_class_cd_L,asset_class_cd_M,asset_class_cd_S,asset_code_1.0,asset_code_2.0,asset_code_4.0,asset_code_19.0,portfolio_type_F,portfolio_type_I,portfolio_type_M,portfolio_type_R,approved
0,2.381248,-0.437057,-0.134588,1.443556,-1.443556,-0.178782,3.652316,-1.670032,-0.195578,-0.300867,-0.21156,-0.022846,-0.933908,0.933908,2.712050,-0.018941,-0.390537,-0.393692,-0.184527,-0.218531,-0.266855,-0.343731,-0.072414,-0.391424,-0.148524,-0.36724,-0.182725,-0.234133,-0.838314,1.688302,-0.174761,-0.414548,-0.343731,-0.218613,-0.291556,-0.490076,-0.171766,0.665466,-0.079368,-0.088806,...,-0.17975,-0.008075,-0.05209,-0.05485,-0.062672,-0.024897,-0.018941,-0.061079,-0.026791,-0.017132,-0.439158,-0.168725,-0.03959,-0.00989,-0.008075,-0.030228,-0.008075,-0.025544,-0.008075,-0.00989,-0.049178,-0.023549,-0.047827,-0.017132,-0.018059,-0.134499,-0.073319,-0.048507,-0.059167,-0.042384,-0.485723,0.181291,-0.163228,-0.067228,-0.03703,-0.018059,-0.787528,-0.159398,0.829996,0
1,1.245663,0.344076,-0.134588,-0.692734,0.692734,-0.178782,-0.273799,0.598791,-0.195578,-0.300867,-0.21156,-0.022846,1.070769,-1.070769,-0.368725,-0.018941,-0.390537,-0.393692,-0.184527,-0.218531,3.747355,-0.343731,-0.072414,-0.391424,-0.148524,-0.36724,-0.182725,-0.234133,1.192871,-0.592311,-0.174761,-0.414548,-0.343731,-0.218613,-0.291556,-0.490076,-0.171766,0.665466,-0.079368,-0.088806,...,-0.17975,-0.008075,-0.05209,-0.05485,-0.062672,-0.024897,-0.018941,-0.061079,-0.026791,-0.017132,-0.439158,-0.168725,-0.03959,-0.00989,-0.008075,-0.030228,-0.008075,-0.025544,-0.008075,-0.00989,-0.049178,-0.023549,-0.047827,-0.017132,-0.018059,-0.134499,-0.073319,-0.048507,-0.059167,-0.042384,2.058786,0.181291,-0.163228,-0.067228,-0.03703,-0.018059,1.269797,-0.159398,-1.204825,0
2,-1.252624,0.344076,-0.134588,1.443556,-1.443556,-0.178782,-0.273799,0.598791,-0.195578,-0.300867,-0.21156,-0.022846,-0.933908,0.933908,-0.368725,-0.018941,-0.390537,-0.393692,-0.184527,-0.218531,-0.266855,-0.343731,13.809417,-0.391424,-0.148524,-0.36724,-0.182725,-0.234133,-0.838314,1.688302,-0.174761,-0.414548,-0.343731,-0.218613,-0.291556,-0.490076,-0.171766,0.665466,-0.079368,-0.088806,...,-0.17975,-0.008075,-0.05209,-0.05485,-0.062672,-0.024897,-0.018941,-0.061079,-0.026791,-0.017132,2.277085,-0.168725,-0.03959,-0.00989,-0.008075,-0.030228,-0.008075,-0.025544,-0.008075,-0.00989,-0.049178,-0.023549,-0.047827,-0.017132,-0.018059,-0.134499,-0.073319,-0.048507,-0.059167,-0.042384,-0.485723,0.181291,-0.163228,-0.067228,-0.03703,-0.018059,1.269797,-0.159398,-1.204825,0
3,1.397075,-0.046491,0.772987,-0.692734,0.692734,-0.17

## __Classification__

Klasifikasi menggunakan xgboost classifier

In [ ]:
X = df_preprocess.drop('approved', axis='columns')
y = df_preprocess['approved']

24537 6135


In [ ]:
xgb_model = XGBClassifier(random_state=0, n_estimators=900)
xgb_model.fit(X, y)

XGBClassifier(n_estimators=900)

# __Melakukan Testing Dari Model__

Membaca data test

In [ ]:
df_test = pd.read_csv('test.csv').iloc[:,1:]

df_test.head()

,gender,date_of_birth,workclass,education_level,education_num,marital_status,occupation,relationship,capital_gain,capital_loss,hours_per_week,address,email,inquiry_purpose_code,institute_type,account_type,asset_class_cd,asset_code,portfolio_type
0,Male,1996-12-31,Private,Some-college,10.0,Never-married,Exec-managerial,Not-in-family,0.0,0.0,40.0,"4310 Carr Keys Apt. 547\r\nNicoletown, MA 99102",krobinson@miller.com,13.0,PVT,47,S,1.0,I
1,Male,1988-01-03,Private,HS-grad,9.0,Married-civ-spouse,Handlers-cleaners,Husband,0.0,0.0,38.0,"PSC 5755, Box 6774\r\nAPO AA 51628",jillwood@yahoo.com,99.0,PVT,5,NaN,1.0,R
2,Male,1970-01-07,Local-gov,Masters,14.0,Married-civ-spouse,Prof-specialty,Husband,0.0,0.0,45.0,USS Spence\r\nFPO AA 93452,mhobbs@hotmail.com,99.0,PVT,189,NaN,1.0,I
3,Male,1995-01-01,Private,HS-grad,9.0,Married-civ-spouse,Craft-repair,Husband,0.0,0.0,40.0,"141 Dennis Bypass\r\nWest Tony, NC 22819",thill@gmail.com,99.0,PVT,5,NaN,1.0,R
4,Female,1983-01-04,Private,HS-grad,9.0,Divorced,Other-service,Not-in-family,0.0,0.0,30.0,"384 Christopher Pine\r\nLeemouth, OR 39895",ocarlson@yahoo.com,13.0,PVT,189,S,1.0,I


Mengaplikasikan semua fungsi preprocessing

In [ ]:
data_wrangling(df_test, train=False)

feature_selection(df_test)
feature_extraction(df_test)

df_test = encoded(df_test)

Menangani hasil categorical encoding yang tidak sama dengan data train

In [ ]:
missing_columns = set((df_preprocess.drop('approved', axis='columns')).columns) - set(df_test.columns)

for i in missing_columns:
  df_test[i] = 0

df_test = df_test[(df_preprocess.drop('approved', axis='columns')).columns]

print(df_preprocess.columns[95], df_test.columns[95])

account_type_222 account_type_222


In [ ]:
len(df_test.columns), len(df_preprocess.columns)

(116, 117)

Melakukan normalisasi

In [ ]:
df_test = normalized(df_test, train=False)

df_test.head()

,age_(day),education_num,capital_total,gender_Female,gender_Male,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital_status_Couple,marital_status_Single,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,hours_per_week_between 40 and 45,hours_per_week_between 45 and 60,hours_per_week_between 60 and 80,hours_per_week_less than 40,hours_per_week_more than 80,inquiry_purpose_code_1.0,...,account_type_172,account_type_173,account_type_175,account_type_176,account_type_177,account_type_178,account_type_179,account_type_181,account_type_184,account_type_185,account_type_187,account_type_189,account_type_191,account_type_195,account_type_197,account_type_198,account_type_213,account_type_220,account_type_221,account_type_222,account_type_223,account_type_224,account_type_226,account_type_227,account_type_228,account_type_240,account_type_999,asset_class_cd_B,asset_class_cd_D,asset_class_cd_L,asset_class_cd_M,asset_class_cd_S,asset_code_1.0,asset_code_2.0,asset_code_4.0,asset_code_19.0,portfolio_type_F,portfolio_type_I,portfolio_type_M,portfolio_type_R
0,-1.176918,-0.046491,-0.134588,-0.692734,0.692734,-0.178782,-0.273799,0.598791,-0.195578,-0.300867,-0.21156,-0.022846,-0.933908,0.933908,-0.368725,-0.018941,-0.390537,2.540058,-0.184527,-0.218531,-0.266855,-0.343731,-0.072414,-0.391424,-0.148524,-0.36724,-0.182725,-0.234133,-0.838314,1.688302,-0.174761,-0.414548,-0.343731,-0.218613,-0.291556,-0.490076,-0.171766,0.665466,-0.079368,-0.088806,...,-0.040811,-0.17975,-0.008075,-0.05209,-0.05485,-0.062672,-0.024897,-0.018941,-0.061079,-0.026791,-0.017132,-0.439158,-0.168725,-0.03959,-0.00989,-0.008075,-0.030228,-0.008075,-0.025544,-0.008075,-0.00989,-0.049178,-0.023549,-0.047827,-0.017132,-0.018059,-0.134499,-0.073319,-0.048507,-0.059167,-0.042384,2.058786,0.181291,-0.163228,-0.067228,-0.03703,-0.018059,1.269797,-0.159398,-1.204825
1,-0.495567,-0.437057,-0.134588,-0.692734,0.692734,-0.178782,-0.273799,0.598791,-0.195578,-0.300867,-0.21156,-0.022846,1.070769,-1.070769,-0.368725,-0.018941,-0.390537,-0.393692,-0.184527,4.576015,-0.266855,-0.343731,-0.072414,-0.391424,-0.148524,-0.36724,-0.182725,-0.234133,1.192871,-0.592311,-0.174761,-0.414548,-0.343731,-0.218613,-0.291556,-0.490076,-0.171766,0.665466,-0.079368,-0.088806,...,-0.040811,-0.17975,-0.008075,-0.05209,-0.05485,-0.062672,-0.024897,-0.018941,-0.061079,-0.026791,-0.017132,-0.439158,-0.168725,-0.03959,-0.00989,-0.008075,-0.030228,-0.008075,-0.025544,-0.008075,-0.00989,-0.049178,-0.023549,-0.047827,-0.017132,-0.018059,-0.134499,-0.073319,-0.048507,-0.059167,-0.042384,-0.485723,0.181291,-0.163228,-0.067228,-0.03703,-0.018059,-0.787528,-0.159398,0.829996
2,0.867135,1.515775,-0.134588,-0.692734,0.692734,-0.178782,3.652316,-1.670032,-0.195578,-0.300867,-0.21156,-0.022846,1.070769,-1.070769,-0.368725,-0.018941,-0.390537,-0.393692,-0.184527,-0.218531,-0.266855,-0.343731,-0.072414,2.554774,-0.148524,-0.36724,-0.182725,-0.234133,1.192871,-0.592311,-0.174761,-0.414548,-0.343731,-0.218613,3.429875,-0.490076,-0.171766,-1.502706,-0.079368,-0.088806,...,-0.040811,-0.17975,-0.008075,-0.05209,-0.05485,-0.062672,-0.024897,-0.018941,-0.061079,-0.026791,-0.017132,2.277085,-0.168725,-0.03959,-0.00989,-0.008075,-0.030228,-0.008075,-0.025544,-0.008075,-0.00989,-0.049178,-0.023549,-0.047827,-0.017132,-0.018059,-0.134499,-0.073319,-0.048507,-0.059167,-0.042384,-0.485723,0.181291,-0.163228,-0.067228,-0.03703,-0.018059,1.269797,-0.159398,-1.204825
3,-1.025507,-0.437057

Memprediksi data test berdasarkan model yang telah dibuat sebelumnya

In [ ]:
test_approved = xgb_model.predict(df_test)

print(f'Hasil prediksi data tes: {test_approved}')
print(f'Perbandingannya : {Counter(test_approved)}')

Hasil prediksi data tes: [0 0 1 ... 0 0 0]
Perbandingannya : Counter({0: 10643, 1: 2924})


In [ ]:
df_test['approved'] = test_approved

df_test.head()

,age_(day),education_num,capital_total,gender_Female,gender_Male,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital_status_Couple,marital_status_Single,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,hours_per_week_between 40 and 45,hours_per_week_between 45 and 60,hours_per_week_between 60 and 80,hours_per_week_less than 40,hours_per_week_more than 80,inquiry_purpose_code_1.0,...,account_type_173,account_type_175,account_type_176,account_type_177,account_type_178,account_type_179,account_type_181,account_type_184,account_type_185,account_type_187,account_type_189,account_type_191,account_type_195,account_type_197,account_type_198,account_type_213,account_type_220,account_type_221,account_type_222,account_type_223,account_type_224,account_type_226,account_type_227,account_type_228,account_type_240,account_type_999,asset_class_cd_B,asset_class_cd_D,asset_class_cd_L,asset_class_cd_M,asset_class_cd_S,asset_code_1.0,asset_code_2.0,asset_code_4.0,asset_code_19.0,portfolio_type_F,portfolio_type_I,portfolio_type_M,portfolio_type_R,approved
0,-1.176918,-0.046491,-0.134588,-0.692734,0.692734,-0.178782,-0.273799,0.598791,-0.195578,-0.300867,-0.21156,-0.022846,-0.933908,0.933908,-0.368725,-0.018941,-0.390537,2.540058,-0.184527,-0.218531,-0.266855,-0.343731,-0.072414,-0.391424,-0.148524,-0.36724,-0.182725,-0.234133,-0.838314,1.688302,-0.174761,-0.414548,-0.343731,-0.218613,-0.291556,-0.490076,-0.171766,0.665466,-0.079368,-0.088806,...,-0.17975,-0.008075,-0.05209,-0.05485,-0.062672,-0.024897,-0.018941,-0.061079,-0.026791,-0.017132,-0.439158,-0.168725,-0.03959,-0.00989,-0.008075,-0.030228,-0.008075,-0.025544,-0.008075,-0.00989,-0.049178,-0.023549,-0.047827,-0.017132,-0.018059,-0.134499,-0.073319,-0.048507,-0.059167,-0.042384,2.058786,0.181291,-0.163228,-0.067228,-0.03703,-0.018059,1.269797,-0.159398,-1.204825,0
1,-0.495567,-0.437057,-0.134588,-0.692734,0.692734,-0.178782,-0.273799,0.598791,-0.195578,-0.300867,-0.21156,-0.022846,1.070769,-1.070769,-0.368725,-0.018941,-0.390537,-0.393692,-0.184527,4.576015,-0.266855,-0.343731,-0.072414,-0.391424,-0.148524,-0.36724,-0.182725,-0.234133,1.192871,-0.592311,-0.174761,-0.414548,-0.343731,-0.218613,-0.291556,-0.490076,-0.171766,0.665466,-0.079368,-0.088806,...,-0.17975,-0.008075,-0.05209,-0.05485,-0.062672,-0.024897,-0.018941,-0.061079,-0.026791,-0.017132,-0.439158,-0.168725,-0.03959,-0.00989,-0.008075,-0.030228,-0.008075,-0.025544,-0.008075,-0.00989,-0.049178,-0.023549,-0.047827,-0.017132,-0.018059,-0.134499,-0.073319,-0.048507,-0.059167,-0.042384,-0.485723,0.181291,-0.163228,-0.067228,-0.03703,-0.018059,-0.787528,-0.159398,0.829996,0
2,0.867135,1.515775,-0.134588,-0.692734,0.692734,-0.178782,3.652316,-1.670032,-0.195578,-0.300867,-0.21156,-0.022846,1.070769,-1.070769,-0.368725,-0.018941,-0.390537,-0.393692,-0.184527,-0.218531,-0.266855,-0.343731,-0.072414,2.554774,-0.148524,-0.36724,-0.182725,-0.234133,1.192871,-0.592311,-0.174761,-0.414548,-0.343731,-0.218613,3.429875,-0.490076,-0.171766,-1.502706,-0.079368,-0.088806,...,-0.17975,-0.008075,-0.05209,-0.05485,-0.062672,-0.024897,-0.018941,-0.061079,-0.026791,-0.017132,2.277085,-0.168725,-0.03959,-0.00989,-0.008075,-0.030228,-0.008075,-0.025544,-0.008075,-0.00989,-0.049178,-0.023549,-0.047827,-0.017132,-0.018059,-0.134499,-0.073319,-0.048507,-0.059167,-0.042384,-0.485723,0.181291,-0.163228,-0.067228,-0.03703,-0.018059,1.269797,-0.159398,-1.204825,1
3,-1.025507,-0.437057,-0.134588,-0.692734,0.692734,-0

Membuat dataframe baru untuk pengumpulan, kemudian mengubah sesuai ketentuan pengumpulan

In [ ]:
df_kumpul = df_test.copy()

df_kumpul = df_kumpul['approved'].reset_index()
df_kumpul.to_csv('hasil_kelompok_21.csv', index=False)